In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestCentroid
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from gensim.models.word2vec import Word2Vec

In [3]:
def get_corpus_dfs(n):
    if n == 1:
        s = 'corpus1'
    elif n == 2:
        s = 'split_corpus2'
    elif n ==3:
        s = 'split_corpus3'

    train = pd.read_csv('data/{}_train.labels'.format(s),
                        delim_whitespace=True,
                        names=['path', 'clf'])
    
    test = pd.read_csv('data/{}_test.labels'.format(s),
                       delim_whitespace=True,
                       names=['path', 'clf'])
    
    train.loc[:, 'path'] = train.loc[:, 'path'].map(lambda s: 'data' + s[1:])
    test.loc[:, 'path'] = test.loc[:, 'path'].map(lambda s: 'data' + s[1:])
    
    return train, test

In [22]:
def rocchio_performance(X_train, y_train, X_test, y_test):
    '''
    Given a train and test split, measure the overall accuracy,
    precision, recall, F-1 score and support of the Rocchio classifier.
    '''
    rocchio_tfidf = NearestCentroid().fit(X_train, y_train)
    
    predictions = rocchio_tfidf.predict(X_test)
    
    acc =  accuracy_score(y_test, predictions)
    prfs = np.vstack(precision_recall_fscore_support(predictions, y_test))
    
    print(acc)
    print()
    print(prfs)
    
    return acc, prfs

## Rocchio-tfidf

In [4]:
train, test = get_corpus_dfs(1)

In [5]:
vec = TfidfVectorizer(input='filename',
                      strip_accents='unicode',
                      stop_words='english',
                      max_df=0.90,
                      min_df=2,
                      norm='l2')

tfidf_train = vec.fit_transform(train.loc[:, 'path'])
clf_train = train.loc[:, 'clf']

In [6]:
tfidf_test = vec.transform(test.loc[:, 'path'])
clf_test = test.loc[:, 'clf']

In [27]:
rocchio_performance(tfidf_train, clf_train, tfidf_test, clf_test);

0.8442437923250564

[[  0.74         0.97752809   0.52         0.79166667   0.91111111]
 [  0.94871795   0.90625      0.8125       0.88372093   0.75460123]
 [  0.83146067   0.94054054   0.63414634   0.83516484   0.82550336]
 [ 39.          96.          16.         129.         163.        ]]


## Rocchio-glove

## Rocchio-word2vec

In [2]:
w2v = Word2Vec.load('embeddings/w2v.corpus1.300d')